<a href="https://colab.research.google.com/github/Apeksha-Jain11/Road_segmentation/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()
from tensorflow.keras.metrics import MeanIoU
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import random
import keras
from keras import callbacks

In [6]:
seed=24
batch_size= 8
n_classes=2

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from tensorflow.keras.utils import to_categorical

#Use this to preprocess input for transfer learning
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

#Define a function to perform  scale images, convert masks to categorical, etc. 
def preprocess_data(img, mask, num_class):

    img = scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape) #Data normalization using Min Max
    img = preprocess_input(img)  #Preprocess based on the pretrained backbone...
    #Convert mask to one-hot
    
    mask[mask==255]=1
    mask = to_categorical(mask, num_class)
      
    return (img,mask)

#Define the generator.
#Not doing rotation and zoom to avoid interpolation interpolation during the process
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
def trainGenerator(train_img_path, train_mask_path, num_class):
    # Data Augmentation
    img_data_gen_args = dict(horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')
    
    image_datagen = ImageDataGenerator(**img_data_gen_args)
    mask_datagen = ImageDataGenerator(**img_data_gen_args)
    
    image_generator = image_datagen.flow_from_directory(
        train_img_path,
        class_mode = None,
        batch_size = batch_size,
        seed = 24)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_mask_path,
        class_mode = None,
        color_mode = 'grayscale',
        batch_size = batch_size,
        seed = 24)
    
    train_generator = zip(image_generator, mask_generator)
    
    for (img, mask) in train_generator:
        img, mask = preprocess_data(img, mask, num_class)
        yield (img, mask)


train_img_path = "/content/drive/MyDrive/road_segmentation_ideal/training/input/training_testing_data/data/train_images/"
train_mask_path = "/content/drive/MyDrive/road_segmentation_ideal/training/input/training_testing_data/data/train_masks/"
train_img_gen = trainGenerator(train_img_path, train_mask_path, num_class=2)

val_img_path = "/content/drive/MyDrive/road_segmentation_ideal/training/input/training_testing_data/data/val_images/"
val_mask_path = "/content/drive/MyDrive/road_segmentation_ideal/training/input/training_testing_data/data/val_masks/"
val_img_gen = trainGenerator(val_img_path, val_mask_path, num_class=2)

In [ ]:
#Sanity check for data and label correspondence
x, y = train_img_gen.__next__()
print(x.shape)
print(y.shape)

for i in range(0,3):
    image = x[i]
    mask = np.argmax(y[i], axis=2)
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.show()

x_val, y_val = val_img_gen.__next__()

for i in range(0,1):
    image = x_val[i]
    mask = np.argmax(y_val[i], axis=2)
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.show()

In [ ]:
num_train_imgs = len(os.listdir('/content/drive/MyDrive/road_segmentation_ideal/training/input/training_testing_data/data/train_images/train/'))
num_val_images = len(os.listdir('/content/drive/MyDrive/road_segmentation_ideal/training/input/training_testing_data/data/val_images/val/'))
steps_per_epoch = num_train_imgs//batch_size
val_steps_per_epoch = num_val_images//batch_size
n_classes=2
file_path= '/content/drive/MyDrive/road_segmentation_ideal/training/input/Model/road_segmentation_25_epochs_RESNET_backbone_batch16.h5'


#Use transfer learning using resnet-34 as pretrained encoder in the U-Net
# I have used segmenataion-models library to get pretrained weights from Imagenet for encoder

################################################################
#Define the model
# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet',
                input_shape=(256, 256, 3),classes=n_classes, activation='softmax')

modelCheck = callbacks.ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only= False, mode='auto', period=1)
model.compile('Adam', loss=sm.losses.categorical_focal_jaccard_loss, metrics=[sm.metrics.iou_score])



#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)
print(model.summary())
print(model.input_shape)
#Fit the model
#history = model.fit(my_generator, validation_data=validation_datagen, steps_per_epoch=len(X_train) // 16, validation_steps=len(X_train) // 16, epochs=100)
#Train the model.
history=model.fit(train_img_gen,
          steps_per_epoch=steps_per_epoch,
          epochs=25,
          verbose=1,
          validation_data=val_img_gen,
          validation_steps=val_steps_per_epoch
          ,callbacks=[modelCheck])



##################################################################
#plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['iou_score']
val_acc = history.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IoU')
plt.plot(epochs, val_acc, 'r', label='Validation IoU')
plt.title('Training and validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.show()


Instructions for updating:
Colocations handled automatically by placer.
Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 256, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d_204 (ZeroPaddin  (None, 262, 262, 3)  0          ['bn_data[0][0]']                
 g2D)                                                                                             
                   

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


1917/1917 [==============================] - 4344s 2s/step - batch: 958.0000 - size: 8.0000 - loss: 0.2229 - iou_score: 0.7920 - val_loss: 0.2276 - val_iou_score: 0.7878
Epoch 2/25
1917/1917 [==============================] - ETA: 0s - batch: 958.0000 - size: 7.9979 - loss: 0.2223 - iou_score: 0.7925

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


1917/1917 [==============================] - 962s 500ms/step - batch: 958.0000 - size: 7.9979 - loss: 0.2223 - iou_score: 0.7925 - val_loss: 0.2303 - val_iou_score: 0.7850
Epoch 3/25
1917/1917 [==============================] - ETA: 0s - batch: 958.0000 - size: 7.9979 - loss: 0.2210 - iou_score: 0.7938

...

1917/1917 [==============================] - 933s 487ms/step - batch: 958.0000 - size: 7.9979 - loss: 0.2194 - iou_score: 0.7953 - val_loss: 0.2227 - val_iou_score: 0.7921
Epoch 6/25
1917/1917 [==============================] - ETA: 0s - batch: 958.0000 - size: 7.9979 - loss: 0.2194 - iou_score: 0.7953

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


1917/1917 [==============================] - 939s 490ms/step - batch: 958.0000 - size: 7.9979 - loss: 0.2194 - iou_score: 0.7953 - val_loss: 0.2249 - val_iou_score: 0.7900
Epoch 7/25
1917/1917 [==============================] - ETA: 0s - batch: 958.0000 - size: 7.9979 - loss: 0.2183 - iou_score: 0.7963

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


1917/1917 [==============================] - 939s 490ms/step - batch: 958.0000 - size: 7.9979 - loss: 0.2183 - iou_score: 0.7963 - val_loss: 0.2242 - val_iou_score: 0.7906
Epoch 8/25
1621/1917 [========================>.....] - ETA: 2:14 - batch: 810.0000 - size: 7.9975 - loss: 0.2174 - iou_score: 0.7971

In [ ]:
%pip install patchify
%pip install -U segmentation-models

     |████████████████████████████████| 50 kB 5.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
